# PCSP 2020 – Limpieza y guardado\n
Este notebook lee `data/raw/contratos_2020_detalle.csv`, lo limpia y guarda un CSV final en `data/processed/pcsp_contracts_2020_clean.csv`.

In [1]:
import pandas as pd
from pathlib import Path

RAW = Path('data/raw/contratos_2020_detalle.csv')
OUT = Path('data/processed/pcsp_contracts_2020_clean.csv')
OUT.parent.mkdir(parents=True, exist_ok=True)

# Manejo cuando falta el archivo: buscar archivos similares y mostrar opciones útiles
if not RAW.exists():
	candidates = list(RAW.parent.glob('*contratos*'))
	if candidates:
		RAW = candidates[0]
		print(f"Archivo original no encontrado; usando: {RAW}")
	else:
		files = list(RAW.parent.glob('*'))
		if files:
			print("No se encontró 'contratos_2020_detalle.csv' en data/raw. Archivos encontrados:")
			for f in files:
				print(" -", f.name)
		else:
			print("No se encontraron archivos en data/raw. Coloque el CSV en ese directorio o descargue los datos.")
		raise FileNotFoundError(f"{RAW} not found")

df = pd.read_csv(RAW)
df.head()

No se encontraron archivos en data/raw. Coloque el CSV en ese directorio o descargue los datos.


FileNotFoundError: data/raw/contratos_2020_detalle.csv not found

In [ ]:
# Validación mínima
expected = {'contract_folder_id','date_2020','cpv_main','cpv_all','source_file'}
missing = expected - set(df.columns)
missing

In [ ]:
# Limpieza de cpv_all: quitar duplicados preservando orden
def uniq_semicolon(s):
    if pd.isna(s) or not str(s).strip():
        return ''
    items = [x.strip() for x in str(s).split(';') if x.strip()]
    seen = []
    for x in items:
        if x not in seen:
            seen.append(x)
    return ';'.join(seen)

df['cpv_main'] = df['cpv_main'].astype('string')
df['cpv_all'] = df['cpv_all'].apply(uniq_semicolon).astype('string')
df.head()

In [ ]:
# Normalizar fecha y crear campos útiles
df['date_2020'] = pd.to_datetime(df['date_2020'], errors='coerce')
df = df.dropna(subset=['contract_folder_id','date_2020'])
df['year'] = df['date_2020'].dt.year.astype(int)
df['month'] = df['date_2020'].dt.to_period('M').astype(str)

# Deduplicar por contrato (si aparece repetido)
df = df.sort_values(['contract_folder_id','date_2020','source_file'])
df = df.drop_duplicates(subset=['contract_folder_id'], keep='first')

df[['contract_folder_id','date_2020','cpv_main','cpv_all','source_file','year','month']].head()

In [ ]:
# Guardar CSV final
df.to_csv(OUT, index=False, encoding='utf-8')
OUT, df['contract_folder_id'].nunique(), len(df)

In [ ]:
# Vista rápida: top CPV\n
df['cpv_main'].value_counts().head(15)